# **1. Introducción teórica.**


**1.   Aprendizaje supervisado**

Se requiere una columna etiqueatda como parte de los datos, esta columna es lo que el algoritmo utiliza para correguir sus decisiones y puede ser una variable discreta, categórica o continua. En la fase de entrenamiento  esta columna  permite que el algoritmo se valide a si mismo; en la fase de pruebas se utiliza para evaluar la calidad del algoritmo comparando las predicciones de modelo con los resutlados reales. La diferencia entre el tipo de variable define el tipo de tarea que queremos que el modelo resuelva:Clasificación o Regresión.

  **Clasificación**

Consiste en calcular la probabilidad de que un dato pertenezca a categorias discretas mediante el análisis de caraterisiticas de entrada, el resultado es la predicción de la probabilidad  de que el dato permanezca a cada categoria posible. Existen tres tipos de clasificación
  1. Binanio: Cada caracteristica de entrada se clasifica en una de dos clases, ejemplo: si/no, falso verdadero.

  2. Multiclases: Cada caracteristica de entrada es clasificada en una de un conjunto de mas de dos clases.

  3. Multi etiqueta: Cada caracteristica de entrada puede tener varias etiquetas.


  **Algoritmos representativos**
  1. LogisticRegresion
  2. DesicionTreeClassifier
  3. RandonForestClasifier
  4. GBTClasifier
  5. MultilayerPerceptronClassifier
  6. LinearSCV
  7. OneVsRest
  8. NaybesBayes
  9. FMClassifier


  **Regresión**

Tareas conocidas como análisis de regresión, es el proceso de estimar las relaciones entre una o más variables dependientes y una o más variables indenpendientes. Existen tres tipos:
  1. Simple: Solo hay uan variable dependiete y una independiente, el valor para el entrenamiento y otro para predecir.

  2. Multiple: Se tiene una variable dependiente para predecir usando múltiples variables independientes para el entrenamiento y entrada

  3. Multivariado:Existen múltiples variables para predecir con múltiples variables independientes para el entranamiento y la entrada por lo que las variables de entrada  salida son valores númericos.

  **Algoritmos representativos**
  1. AFTSurvival​Re⁠gression.
  2. DecisionTreeRegressor
  3. GBT​Re⁠gressor
  

**2.   Aprendizaje no supervisado**

Se utilizan cuando los datos no tienen etiquetas pero, se necesita encontrar patrones interesantes, predecir comportamientos o calcular semejanzas sin concer el resultado deseado. Esto algoritmos se pueden utilizar con algoritmos de aprendizaje supervisado de manera indistinta como parte del proceso de extracción de carácterisicas. Las tareas cómunes para el uso de algortimos de aprendizaje supervisado son la mineria de patrones frecuentes y la agrupación de clusteres.


  **Mineria de patrones frecuentes**
Se enmarca en la categoria de aprendizaje de reglas de asociación, se base en la identificación de reglas para descubrir relaciones entre variables en los datos. Los algoritmos suelen buscar primero elementos frecuentes en el conjunto de datos y, a continuación, pares o conjuntos de elementos frecuentes, por ejemplo articulos que se comprar juntos con frecuencia.


  **Algoritmos representativos**
  1. FPGrowth
  2. PrefixSpan



  **Clustering**
es una técnica de agrupamiento para descubrir las relaciones ocultas entre los puntos de datos, se utiliza habitualmente para la segmentación de clientes, el procesamiento y la detección de imágenes, el filtrado de spam, la detección de anomalías, etc.

  **Algoritmos representativos**
  1. LDA
  2. GaussianMixture
  3. KMeans
  4. BisectingKMeans
  5. PowerIterationClustering

# **2. Selección de los datos.**

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
#Se prepara entorno de ejecución
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
#spark = SparkSession.builder.master("local[*]").getOrCreate()
spark = SparkSession.builder \
                    .master("local[*]") \
                    .appName("Ejemplo") \
                    .getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [21]:
#Se crea dataframe
df = spark.read.csv('sample_data/Combined_Flights_2022.csv', header=True, sep=",", inferSchema=True)
#Visualización limitada a los primeros 10 registros
df.show(10)

+----------+--------------------+------+----+---------+--------+----------+-------+---------------+--------+-------+---------------+-------+--------------+-----------------+--------+----+-------+-----+----------+---------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+--------------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+-------------------+---------+-------------+-------------+-------+--------+--------------------+----------+-------+---------+--------+------+----------+--------+--------+------------------+----------+-------------+------------------+
|FlightDate|             Airline|Origin|Dest|Cancelled|Diverted|CRSDepTime|DepTime|DepDelayMinutes|De

In [22]:
# Imprimiendo el esquema del Dataframe
df.printSchema()

root
 |-- FlightDate: string (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: boolean (nullable = true)
 |-- Diverted: boolean (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- DepDelayMinutes: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- ArrDelayMinutes: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Marketing_Airline_Network: string (nullable = true)
 |-- Operated_or_Branded_Code_Share_Partners: string (nullable = true)


### Selección de datos por muestre estratificado

In [25]:
#selección de muestra estratificada
stratified_df = df.sampleBy('DepDel15', fractions={1:0.5, 0:0.5}, seed=42)
stratified_df.show(5)

+----------+--------------------+------+----+---------+--------+----------+-------+---------------+--------+-------+---------------+-------+--------------+-----------------+--------+----+-------+-----+----------+---------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+-------------------+---------+-------------+-------------+-------+--------+--------------------+----------+-------+---------+--------+------+----------+--------+--------+------------------+----------+-------------+------------------+
|FlightDate|             Airline|Origin|Dest|Cancelled|Diverted|CRSDepTime|DepTime|DepDelayMinutes|DepDelay

# **3. Preparación de los datos.**

In [26]:
#Se eliminan registros con valores nulos
df_clean = stratified_df.dropna()

#Se eliminan registros duplicados
df_clean = stratified_df.dropDuplicates()

# **4. Preparación del conjunto de entrenamiento y prueba.**

In [27]:
# Particionamiento típico
# Se define el valor por default del número de ejecutores
spark.conf.set("spark.sql.shuffle.partitions", "200")
#Particionamiento de los datos en prueba y test
train_data,test_data = df_clean.randomSplit([0.8,0.2], seed = 42)
print(f"""Existen {train_data.count()} instancias en el conjunto train, y {test_data.count()} en el conjunto test""")

Existen 1584455 instancias en el conjunto train, y 395486 en el conjunto test


In [28]:
# prompt: imprimir el plan de transformación al llamar randomSplit
train_data.explain()
test_data.explain()

== Physical Plan ==
*(2) Sample 0.0, 0.8, false, 42
+- *(2) Sort [FlightDate#10632 ASC NULLS FIRST, Airline#10633 ASC NULLS FIRST, Origin#10634 ASC NULLS FIRST, Dest#10635 ASC NULLS FIRST, Cancelled#10636 ASC NULLS FIRST, Diverted#10637 ASC NULLS FIRST, CRSDepTime#10638 ASC NULLS FIRST, DepTime#10639 ASC NULLS FIRST, DepDelayMinutes#10640 ASC NULLS FIRST, DepDelay#10641 ASC NULLS FIRST, ArrTime#10642 ASC NULLS FIRST, ArrDelayMinutes#10643 ASC NULLS FIRST, AirTime#10644 ASC NULLS FIRST, CRSElapsedTime#10645 ASC NULLS FIRST, ActualElapsedTime#10646 ASC NULLS FIRST, Distance#10647 ASC NULLS FIRST, Year#10648 ASC NULLS FIRST, Quarter#10649 ASC NULLS FIRST, Month#10650 ASC NULLS FIRST, DayofMonth#10651 ASC NULLS FIRST, DayOfWeek#10652 ASC NULLS FIRST, Marketing_Airline_Network#10653 ASC NULLS FIRST, Operated_or_Branded_Code_Share_Partners#10654 ASC NULLS FIRST, DOT_ID_Marketing_Airline#10655 ASC NULLS FIRST, ... 37 more fields], false, 0
   +- *(2) HashAggregate(keys=[OriginStateName#10669,

# **5. Construcción de modelos de aprendizaje supervisado y no supervisado.**

## Algoritmo de aprendizaje supervisado



In [39]:
from pyspark.ml.feature import VectorAssembler

#Se genera assembler para la columna DepDel15
assembler = VectorAssembler(inputCols=['DepDel15'], outputCol = 'Attributes')
output_train = assembler.transform(train_data)
output_test = assembler.transform(test_data)

output_train.show(5)
output_test.show(5)

+----------+--------------------+------+----+---------+--------+----------+-------+---------------+--------+-------+---------------+-------+--------------+-----------------+--------+----+-------+-----+----------+---------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+--------------------+---------+-------------+-------------+-------+--------+--------------------+----------+-------+---------+--------+------+----------+--------+--------+------------------+----------+-------------+------------------+----------+
|FlightDate|             Airline|Origin|Dest|Cancelled|Diverted|CRSDepTime|DepTime|DepDelayMinu

In [41]:
# Se manda a entrenar con un modelo de regresion Lineal
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'Attributes', labelCol = 'DepDel15', maxIter=10, regParam=0.3, elasticNetParam=1.0)
lr_model = lr.fit(output_train)

In [42]:
# Se Imprimen los valores de los coeficientes
print ("The coefficient of the model is : ", lr_model.coefficients)
print ("The Intercept of the model is : ", lr_model.intercept)

The coefficient of the model is :  [1.3983354435712456]
The Intercept of the model is :  -1.6674354880260787


In [43]:
# Se aplica el modelo al conjunto test
Pred_lr = lr_model.evaluate(output_test)

In [44]:
# Evaluación del modelo
from pyspark.ml.evaluation import RegressionEvaluator

#Root Mean Square Error
eval_lr = RegressionEvaluator(labelCol="DepDel15", predictionCol="prediction", metricName="rmse")
rmse_lr = eval_lr.evaluate(Pred_lr.predictions)
print("RMSE: %.3f" % rmse_lr)

# Mean Square Error
mse = eval_lr.evaluate(Pred_lr.predictions, {eval_lr.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval_lr.evaluate(Pred_lr.predictions, {eval_lr.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval_lr.evaluate(Pred_lr.predictions, {eval_lr.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 0.468
MSE: 0.219
MAE: 0.219
r2: -0.280


## Algoritmo de aprendizaje no supervisado

In [46]:
#Se elige modelo GaussianMixture
from pyspark.ml.clustering import GaussianMixture
gm = GaussianMixture(k=7, tol=0.01, seed=42,
                     featuresCol="Attributes", maxIter=10)
model = gm.fit(output_train)

In [51]:
#Objeto resumen
summary = model.summary

In [48]:
#Número de objetos en cada conglomerado
summary.cluster.select("prediction").distinct().count()

2

In [49]:
#Rendimietno del modelo
summary.logLikelihood

22105387.640255135

In [50]:
summary.clusterSizes

[1237783, 0, 0, 346672, 0, 0, 0]